<a href="https://colab.research.google.com/github/monindew/Pytorch_Study/blob/main/20240817.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"monindew","key":"dc0a261c3e94c912b10f2309c0193350"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!pip install kaggle

In [4]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

  0% 0.00/199k [00:00<?, ?B/s]
100% 199k/199k [00:00<00:00, 19.4MB/s]


In [5]:
!unzip house-prices-advanced-regression-techniques.zip

Archive:  house-prices-advanced-regression-techniques.zip
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [6]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [7]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [8]:
print(df_train.shape)
print(df_test.shape)

(1460, 81)
(1459, 80)


In [9]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [10]:
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [11]:
df_train.select_dtypes(include=[np.number]).corr()["SalePrice"][(df_train.select_dtypes(include=[np.number]).corr()["SalePrice"] > 0.5) | (df_train.select_dtypes(include=[np.number]).corr()["SalePrice"] < -0.5)]

,SalePrice
OverallQual,0.790982
YearBuilt,0.522897
YearRemodAdd,0.507101
TotalBsmtSF,0.613581
1stFlrSF,0.605852
GrLivArea,0.708624
FullBath,0.560664
TotRmsAbvGrd,0.533723
GarageCars,0.640409
GarageArea,0.623431


In [12]:
feature_list = df_train.select_dtypes(include=[np.number]).corr()["SalePrice"][(df_train.select_dtypes(include=[np.number]).corr()["SalePrice"] > 0.5) | (df_train.select_dtypes(include=[np.number]).corr()["SalePrice"] < -0.5)].index

In [13]:
df_train[feature_list].isnull().sum()

,0
OverallQual,0
YearBuilt,0
YearRemodAdd,0
TotalBsmtSF,0
1stFlrSF,0
GrLivArea,0
FullBath,0
TotRmsAbvGrd,0
GarageCars,0
GarageArea,0


In [14]:
df_train[feature_list].dtypes

,0
OverallQual,int64
YearBuilt,int64
YearRemodAdd,int64
TotalBsmtSF,int64
1stFlrSF,int64
GrLivArea,int64
FullBath,int64
TotRmsAbvGrd,int64
GarageCars,int64
GarageArea,int64


In [15]:
features = list(feature_list[:-1])
features

['OverallQual',
 'YearBuilt',
 'YearRemodAdd',
 'TotalBsmtSF',
 '1stFlrSF',
 'GrLivArea',
 'FullBath',
 'TotRmsAbvGrd',
 'GarageCars',
 'GarageArea']

In [16]:
for feature in features:
  df_test[feature].fillna(df_test[feature].mean(), inplace=True)

In [17]:
X_train = df_train[features]
y_train = df_train[["SalePrice"]].values
X_test = df_test[features]

In [18]:
print(X_train.shape, X_train.dtypes)

(1460, 10) OverallQual     int64
YearBuilt       int64
YearRemodAdd    int64
TotalBsmtSF     int64
1stFlrSF        int64
GrLivArea       int64
FullBath        int64
TotRmsAbvGrd    int64
GarageCars      int64
GarageArea      int64
dtype: object


In [19]:
std_scaler = StandardScaler()
std_scaler.fit(X_train)
X_train_tensor = torch.from_numpy(std_scaler.transform(X_train)).float()
X_test_tensor = torch.from_numpy(std_scaler.transform(X_test)).float()
y_train_tensor = torch.from_numpy(y_train).float()

In [20]:
print(X_train_tensor.shape, X_test_tensor.shape, y_train_tensor.shape)

torch.Size([1460, 10]) torch.Size([1459, 10]) torch.Size([1460, 1])


In [21]:
nb_epochs = 10000
minibatch_size = 256

In [22]:
class FunModel(nn.Module):
  def __init__(self, input_dim, output_dim):
    super().__init__()

    self.linear_layers = nn.Sequential(
        nn.Linear(input_dim, 20),
        nn.LeakyReLU(),
        nn.Linear(20, 10),
        nn.LeakyReLU(),
        nn.Linear(10, 8),
        nn.LeakyReLU(),
        nn.Linear(8, 6),
        nn.LeakyReLU(),
        nn.Linear(6, output_dim)
    )

  def forward(self, x):
    y = self.linear_layers(x)
    return y

In [23]:
input_dim = X_train_tensor.size(-1)
output_dim = y_train_tensor.size(-1)
print(input_dim, output_dim)
model = FunModel(input_dim, output_dim)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

10 1


In [24]:
indices = torch.randperm(X_train_tensor.size(0))
print(indices)
x_batch_list = torch.index_select(X_train_tensor, 0, indices)
y_batch_list = torch.index_select(y_train_tensor, 0, indices)
x_batch_list = x_batch_list.split(minibatch_size, 0)
y_batch_list = y_batch_list.split(minibatch_size, 0)

tensor([1078,  939,  851,  ...,  330,  247, 1211])


In [29]:
for index in range(nb_epochs):
  indices = torch.randperm(X_train_tensor.size(0))
  x_batch_list = torch.index_select(X_train_tensor, 0, indices)
  y_batch_list = torch.index_select(y_train_tensor, 0, indices)
  x_batch_list = x_batch_list.split(minibatch_size, 0)
  y_batch_list = y_batch_list.split(minibatch_size, 0)

  epoch_loss = list()
  for x_minibatch, y_minibatch in zip(x_batch_list, y_batch_list):
    y_minibatch_pred = model(x_minibatch)
    loss =torch.sqrt(loss_function(y_minibatch_pred, y_minibatch))
    epoch_loss.append(loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (index % 500) == 0:
    print(index, sum(epoch_loss) / len(epoch_loss))

0 tensor(32662.6953, grad_fn=<DivBackward0>)
500 tensor(32643.1797, grad_fn=<DivBackward0>)
1000 tensor(32582.5684, grad_fn=<DivBackward0>)
1500 tensor(32823.4180, grad_fn=<DivBackward0>)
2000 tensor(32398.2402, grad_fn=<DivBackward0>)
2500 tensor(31996.5293, grad_fn=<DivBackward0>)
3000 tensor(32341.3125, grad_fn=<DivBackward0>)
3500 tensor(32563.9609, grad_fn=<DivBackward0>)
4000 tensor(32290.2246, grad_fn=<DivBackward0>)
4500 tensor(32574.8496, grad_fn=<DivBackward0>)
5000 tensor(32886.3633, grad_fn=<DivBackward0>)
5500 tensor(32099.1035, grad_fn=<DivBackward0>)
6000 tensor(31785.1738, grad_fn=<DivBackward0>)
6500 tensor(31431.4707, grad_fn=<DivBackward0>)
7000 tensor(31923.0918, grad_fn=<DivBackward0>)
7500 tensor(32067.2188, grad_fn=<DivBackward0>)
8000 tensor(32338.0527, grad_fn=<DivBackward0>)
8500 tensor(31925.1387, grad_fn=<DivBackward0>)
9000 tensor(31969.2441, grad_fn=<DivBackward0>)
9500 tensor(31498.6387, grad_fn=<DivBackward0>)


In [31]:
y_pred_list = list()
x_test_batch_list = X_test_tensor.split(minibatch_size, 0)
model.eval()
with torch.no_grad():
  for x_minibatch in x_test_batch_list:
    y_minibatch_pred = model(x_minibatch)
    y_pred_list.extend(y_minibatch_pred.squeeze().detach().tolist())
submission = pd.DataFrame({'Id':df_test["Id"], "SalePrice": y_pred_list})
submission.to_csv('submission.csv', index=False)